In [2]:
import pandas as pd
import numpy as np

### Input parameters for the model, loaded as a dataframe:

In [4]:
df = pd.DataFrame({'State' : ['Normal', '+', '++', '+++', '++++']})
df['Ext Diam Sys'] = 6.2 # Ext Diam Sys
df['Ext Diam Dias'] = 6.7 # Ext Diam Dias
df['Ext Long Sys'] = 8.4 # Ext Diam Sys
df['Ext Long Dias'] = 10 # Ext long Dias
df['Int Diam Dias'] = [4.9, 4.6, 4.4, 4.2, 4.0] # Int Diam Dias
# df['Int Diam Dias'] = 4 #[1, 0.9, 0.8, 0.7, 0.6] # Int Diam Dias to break the model
df['LVWT'] = [0.9, 1.1, 1.2, 1.3, 1.4] # LVWT
display(df)
default = df

,State,Ext Diam Sys,Ext Diam Dias,Ext Long Sys,Ext Long Dias,Int Diam Dias,LVWT
0,Normal,6.2,6.7,8.4,10,4.9,0.9
1,+,6.2,6.7,8.4,10,4.6,1.1
2,++,6.2,6.7,8.4,10,4.4,1.2
3,+++,6.2,6.7,8.4,10,4.2,1.3
4,++++,6.2,6.7,8.4,10,4.0,1.4


## Geometric Manipulations:

getParams() executes the iterative geometric manipulations with a given input DataFrame and a function for calculating volume from a given shape. It returns the dataframe with all of the appended calculations on the dataframe

In [9]:
def getParams(df, volFunc):
    '''
    inputs:
        - df : Dataframe with Input parameters
        - volFunc : Function for calculating volume with 2 parameters: (diameter, length) or the 
                    equivalent of (diameter,length)
    outputs:
        - df : Dataframe with added columns
    '''
    df['Int Long Dias'] = df.apply(lambda row: row['Ext Long Dias'] - row['LVWT'], axis = 1)
    df['Ext Vol Dias'] = df.apply(lambda row: volFunc(row['Ext Diam Dias'], row['Ext Long Dias']), axis = 1)
    df['Int Vol Dias'] = df.apply(lambda row: volFunc(row['Int Diam Dias'], row['Int Long Dias']), axis = 1)
    df['Myocardial Vol'] = df['Ext Vol Dias'] - df['Int Vol Dias']
    df['Ext Vol Sys'] = df.apply(lambda row: volFunc(row['Ext Diam Sys'], row['Ext Long Sys']), axis = 1)
    df['Int Vol Sys'] = df.apply(lambda row: row['Ext Vol Sys'] - row['Myocardial Vol'], axis = 1)
    df['Stroke Vol'] = df.apply(lambda row: row['Int Vol Dias'] - row['Int Vol Sys'], axis = 1)
    df['Ejection Frac'] = df.apply(lambda row: row['Stroke Vol']/ row['Int Vol Dias'] * 100, axis = 1)
    
    return df

In [8]:
print("\nDefault:")
volFunc = lambda D, L: (np.pi * (D **2) * L )/ 6 # Default
df1 = getParams(default, volFunc)
display(df1.round(2))

print("\nCone:")
volFunc = lambda D, L: (np.pi * (D **2) * L )/ 12 # Cone
df2 = getParams(default, volFunc)
display(df2.round(2))

print("\nCube:")
volFunc = lambda D, L: D**3
df3 = getParams(default, volFunc)
display(df3.round(2))

print("\nCylinder:")
volFunc = lambda D,L: np.pi * (D/2) **2 * L 
df4 = getParams(default, volFunc)
display(df4.round(2))

print("\nHemiSphere:")
volFunc = lambda D,L: np.pi * (D/2)**3 * (2/3)
df5 = getParams(default, volFunc)
display(df5.round(2))


Default:


,State,Ext Diam Sys,Ext Diam Dias,Ext Long Sys,Ext Long Dias,Int Diam Dias,LVWT,Int Long Dias,Ext Vol Dias,Int Vol Dias,Myocardial Vol,Ext Vol Sys,Int Vol Sys,Stroke Vol,Ejection Frac
0,Normal,6.2,6.7,8.4,10,4.9,0.9,9.1,235.04,114.40,120.64,169.07,48.43,65.98,57.67
1,+,6.2,6.7,8.4,10,4.6,1.1,8.9,235.04,98.61,136.44,169.07,32.63,65.98,66.91
2,++,6.2,6.7,8.4,10,4.4,1.2,8.8,235.04,89.20,145.84,169.07,23.23,65.98,73.96
3,+++,6.2,6.7,8.4,10,4.2,1.3,8.7,235.04,80.36,154.69,169.07,14.38,65.98,82.10
4,++++,6.2,6.7,8.4,10,4.0,1.4,8.6,235.04,72.05,163.00,169.07,6.07,65.98,91.57



Cone:


,State,Ext Diam Sys,Ext Diam Dias,Ext Long Sys,Ext Long Dias,Int Diam Dias,LVWT,Int Long Dias,Ext Vol Dias,Int Vol Dias,Myocardial Vol,Ext Vol Sys,Int Vol Sys,Stroke Vol,Ejection Frac
0,Normal,6.2,6.7,8.4,10,4.9,0.9,9.1,117.52,57.20,60.32,84.53,24.21,32.99,57.67
1,+,6.2,6.7,8.4,10,4.6,1.1,8.9,117.52,49.30,68.22,84.53,16.32,32.99,66.91
2,++,6.2,6.7,8.4,10,4.4,1.2,8.8,117.52,44.60,72.92,84.53,11.61,32.99,73.96
3,+++,6.2,6.7,8.4,10,4.2,1.3,8.7,117.52,40.18,77.34,84.53,7.19,32.99,82.10
4,++++,6.2,6.7,8.4,10,4.0,1.4,8.6,117.52,36.02,81.50,84.53,3.04,32.99,91.57



Cube:


,State,Ext Diam Sys,Ext Diam Dias,Ext Long Sys,Ext Long Dias,Int Diam Dias,LVWT,Int Long Dias,Ext Vol Dias,Int Vol Dias,Myocardial Vol,Ext Vol Sys,Int Vol Sys,Stroke Vol,Ejection Frac
0,Normal,6.2,6.7,8.4,10,4.9,0.9,9.1,300.76,117.65,183.11,238.33,55.21,62.44,53.07
1,+,6.2,6.7,8.4,10,4.6,1.1,8.9,300.76,97.34,203.43,238.33,34.90,62.44,64.14
2,++,6.2,6.7,8.4,10,4.4,1.2,8.8,300.76,85.18,215.58,238.33,22.75,62.44,73.29
3,+++,6.2,6.7,8.4,10,4.2,1.3,8.7,300.76,74.09,226.68,238.33,11.65,62.43,84.27
4,++++,6.2,6.7,8.4,10,4.0,1.4,8.6,300.76,64.00,236.76,238.33,1.56,62.44,97.55



Cylinder:


,State,Ext Diam Sys,Ext Diam Dias,Ext Long Sys,Ext Long Dias,Int Diam Dias,LVWT,Int Long Dias,Ext Vol Dias,Int Vol Dias,Myocardial Vol,Ext Vol Sys,Int Vol Sys,Stroke Vol,Ejection Frac
0,Normal,6.2,6.7,8.4,10,4.9,0.9,9.1,352.57,171.60,180.96,253.6,72.64,98.96,57.67
1,+,6.2,6.7,8.4,10,4.6,1.1,8.9,352.57,147.91,204.66,253.6,48.95,98.96,66.91
2,++,6.2,6.7,8.4,10,4.4,1.2,8.8,352.57,133.81,218.76,253.6,34.84,98.96,73.96
3,+++,6.2,6.7,8.4,10,4.2,1.3,8.7,352.57,120.53,232.03,253.6,21.57,98.96,82.10
4,++++,6.2,6.7,8.4,10,4.0,1.4,8.6,352.57,108.07,244.49,253.6,9.11,98.96,91.57



HemiSphere:


,State,Ext Diam Sys,Ext Diam Dias,Ext Long Sys,Ext Long Dias,Int Diam Dias,LVWT,Int Long Dias,Ext Vol Dias,Int Vol Dias,Myocardial Vol,Ext Vol Sys,Int Vol Sys,Stroke Vol,Ejection Frac
0,Normal,6.2,6.7,8.4,10,4.9,0.9,9.1,78.74,30.80,47.94,62.39,14.45,16.35,53.07
1,+,6.2,6.7,8.4,10,4.6,1.1,8.9,78.74,25.48,53.26,62.39,9.14,16.35,64.14
2,++,6.2,6.7,8.4,10,4.4,1.2,8.8,78.74,22.30,56.44,62.39,5.96,16.35,73.29
3,+++,6.2,6.7,8.4,10,4.2,1.3,8.7,78.74,19.40,59.34,62.39,3.05,16.35,84.27
4,++++,6.2,6.7,8.4,10,4.0,1.4,8.6,78.74,16.76,61.98,62.39,0.41,16.35,97.55
